In [1]:
%%capture --no-stderr
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
import warnings
import nest_asyncio
from IPython.display import display, Markdown

warnings.filterwarnings("ignore")
nest_asyncio.apply()
_ = load_dotenv(find_dotenv())

## Building our RAG workflow

In [3]:
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from phoenix.otel import register
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

## Log traces using Llama Trace
tracer_provider = register(
  project_name="llama-trace-v1",
  endpoint="https://app.phoenix.arize.com/v1/traces"
)
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: llama-trace-v1
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [8]:
Settings.llm = OpenAI("gpt-4o-mini")

## Building our RAG Service

## Bringing Langgraph into the picture